
# Multiplicative Free Convolution

## Goals of this script: 
- I.   Monte-Carlo for Multiplicative Free Convolutions
- II.  Inverting explicit S-transforms
- III. Application to FreeNN


In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt

# I. Monte-Carlo for some explicit multiplicative free convolutions.

Recall that the Marchenko-Pastur law is the universal limit for singular values of Gaussian matrices. Following Marchenko and Pastur (1967)
$$ \frac{1}{2 \pi} \frac{\sqrt{(x-l)(r-x)}}{x} dx ,$$
where 
$$r = (1+\sqrt{c})^2$$
$$l = (1-\sqrt{c})^2$$
and $c$ being the scale parameter.

In [ ]:
# One MP

N=1000
num_bins=50

c = 3 # MP scale parameter
r = (1+np.sqrt(c))**2 #Right end
l = (1-np.sqrt(c))**2 #Left end

G = np.random.normal( size=(N,c*N) )
W = G.dot( G.transpose() )
W = W/N
diag, U = np.linalg.eig(W)

# Histogram of singular values
fig, ax = plt.subplots()
n, bins, patches = ax.hist(diag, num_bins, density=True)
y = np.sqrt( (r-bins)*(bins-l) )/(2*np.pi*bins)
ax.plot(bins, y, '--', linewidth=4)
ax.set_xlabel('Eigenvalues')
ax.set_ylabel('Probability density')
ax.set_title(r'Histogram of singular values for N={}'.format(N))
fig.tight_layout()
plt.xlim(0,r+0.5)
plt.show()

In [ ]:
# Multiple ones
scale_params = [3, 0.5, 2]

N=1000
num_bins=50

M = np.identity(N)
for c in scale_params:
    p, q = M.shape
    qq = int(c*q)
    G = np.random.normal( size=(q,qq) )/np.sqrt(q+qq)
    M = M.dot( G )
    W = M.dot( M.transpose() )
    diag, U = np.linalg.eig(W)

    # Histogram of singular values
    fig, ax = plt.subplots()
    n, bins, patches = ax.hist(diag, num_bins)
    #y = np.sqrt( (r-bins)*(bins-l) )/(2*np.pi*bins)
    #ax.plot(bins)
    ax.set_xlabel('Eigenvalues')
    ax.set_ylabel('Probability density')
    ax.set_title(r'Histogram of singular values for N={}'.format(N))
    fig.tight_layout()
    plt.show()

# II. Inverting explicit S-transforms
Here we consider the $S$-transforms of Marchenko-Pastur distributions, which are of the form:
$$ S_{W_l}(z) = \frac{1}{\sigma_l} \frac{1}{1+\lambda_l z} \ ,$$
where the $\lambda_i$ are the scale pameters and $\sigma_l^2$ are the variances. For simplicity, we assume for now $\sigma_l^2=1$.

## 1. Basic definitions

In [ ]:
import time
import numpy as np
from freenn.core import newton, adaptative

# Array of \lambda_l's
scale_params = [3, 1, 0.5]
# Array of \Lambda_l / \lambda_l used for scaling z (Check error in paper!)
w_scaling    = np.cumprod(scale_params)/scale_params

# Compute coefficients of M_inverse (numerator and denominator)
# Conventions:
#   - Coefficients are numpy arrays
#   - Highest degree comes first
#
# Of numerator of M_inverse
roots         = np.append(-1, -1/w_scaling)
leading_coeff = np.prod(w_scaling)
coeff_num     = np.poly( roots ) * leading_coeff
# Of denominator of M_inverse = w
coeff_den     = np.array( [1, 0] )

wrapper = newton.Polynomial_Kantorovich_Wrapper( coeff_num, coeff_den)

## 2. Newton-Raphson iterations

Important formulae
$$ m = zg - 1$$

In [ ]:
# Test for a value in basin of attraction
j = complex(0,1)
z = 1+ j*3
print("z: ", z)
m = newton.newton_raphson   ( z, function_wrapper=wrapper )
g = newton.newton_raphson_ZG( z, function_wrapper = wrapper)
print("M(z)         = ", m )
print("G(z)         = ", g )
print("M_inverse(m) = ", wrapper.phi(m)   )
print("Error        = ", abs(z-wrapper.phi(m)) )

In [ ]:
# Test the Kantorovich criterion for basins of attraction
z = complex(1,10)
print( newton.is_in_basin_ZG(z, 1/z, function_wrapper = wrapper, debug=True ) )
z = complex(1,5)
print( newton.is_in_basin_ZG(z, 1/z, function_wrapper = wrapper, debug=True ) )

In [ ]:
j       = complex(0,1)
t       = 1
debug   = False
z_array = 1+ j*np.array([ 1, 0.1, 1e-5, 1e-8, 1e-10])
proxy   = None
for z in z_array:
    print("z: ", z)
    adaptative.reset_counters()
    g = adaptative.compute_G_adaptative( z, function_wrapper = wrapper, proxy=proxy, debug=False)
    proxy = (z, g)
    print("G(z)  = ", g )
    z_check = wrapper.phi(z*g-1)
    print("Error = ", abs(z_check-z) )
    print("Number of calls to NR: ", adaptative.call_counter_NR)
    print("Number of calls to attraction basin test: ", adaptative.call_counter_failed_basin)
    print("")


## 3. Computation of the measure (multiple passes)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#N: Space mesh
N=100
a=-2
b=10

#Init
space_grid = np.linspace(a, b, N)
dx = (b-a)/N

#Multiple passes for the number of iterations
imaginary_parts = [1.0, 0.1, 0.01, 1e-4, 1e-8]
densities       = []
hilbert_transf  = []
pass_counter    = 0
iter_count      = [ [] for i in space_grid ]
errors1         = [ [] for i in space_grid ]
errors2         = [ [] for i in space_grid ]
choices         = [ [] for i in space_grid ]

j       = complex(0,1)
fig = plt.figure( figsize = (12,7) )
ax = fig.add_subplot( 111 )
y_proxy = None
guess   = None
G       = np.array( space_grid + complex(0,1) )
for y in imaginary_parts:
    start = time.time()
    adaptative.reset_counters()
    # Compute
    z = np.array( space_grid + y*complex(0,1) )
    for i in range(N):
        if y_proxy is None:
            G[i] = adaptative.compute_G_adaptative(z[i], function_wrapper = wrapper, proxy=None)
        else:
            guess = (z[i].real+j*y_proxy, G[i])
            G[i] = adaptative.compute_G_adaptative(z[i], function_wrapper = wrapper, proxy=guess)
    # Statistics
    pass_counter += 1
    timing        = time.time() - start
    print ('Pass [{}/{}], Duration: {:.1f} ms' 
           .format(pass_counter, len(imaginary_parts), 1000*timing))
    print("Number of calls to subroutine:")
    print("'Newton-Raphson'  :", adaptative.call_counter_NR)
    print("'Attraction basin':", adaptative.call_counter_failed_basin)
    print("")
    # Plot
    ax.plot(space_grid, -np.imag(G)/np.pi, '--', label="y=%.5f"%y)
    ax.set(xlabel='Space (x)', ylabel='Value',
           title='Density')
    ax.grid()
    #
    y_proxy = y
plt.ylim(0,0.2)
plt.legend()
plt.show()
